In [19]:
# Dependencies
from bs4 import BeautifulSoup as bs
import requests
from splinter import Browser
import os
import pandas as pd
import time
from webdriver_manager.chrome import ChromeDriverManager

In [24]:
# Splinter
executable_path = {'executable_path': ChromeDriverManager().install()}
browser = Browser('chrome', **executable_path, headless=False)

[WDM] - ====== WebDriver manager ======
[WDM] - Current google-chrome version is 89.0.4389
[WDM] - Get LATEST driver version for 89.0.4389


[WDM] - Driver [C:\Users\jglen\.wdm\drivers\chromedriver\win32\89.0.4389.23\chromedriver.exe] found in cache


In [29]:
# Visit NASA
url = 'https://redplanetscience.com/'
browser.visit(url)
time.sleep(1)

In [34]:
nasa_html = browser.html
soup = bs(nasa_html, 'html.parser')

In [40]:
nasa_headline = soup.find('div', class_='content_title').get_text()
nasa_teaser = soup.find('div', class_='article_teaser_body').get_text()
print(nasa_headline)
print(nasa_teaser)

NASA's Mars 2020 Rover Goes Coast-to-Coast to Prep for Launch
The agency's first step in returning rocks from Mars just arrived at Kennedy Space Center. The Mars 2020 team now begins readying for a launch to the Red Planet this July.


In [ ]:
# visit the JPL website and scrape the featured image
jpl_url = 'https://spaceimages-mars.com'
browser.visit(jpl_url)
time.sleep(1)
browser.click_link_by_partial_text('FULL IMAGE')
soup = bs(browser.html, 'html.parser')

In [ ]:
image_src = soup.find_all('img')[1]["src"]
featured_image_url = jpl_url + image_src

In [48]:
# visit Mars Facts and scrape the Mars Facts table
mars_facts_url = 'https://galaxyfacts-mars.com'
browser.visit(mars_facts_url)
time.sleep(1)
mars_facts_soup = bs(browser.html, 'html.parser')

In [50]:
mars_df = pd.read_html(browser.html)[1]
mars_facts_html = mars_df.to_html

ValueError: No tables found

In [49]:
# Mars Hemispheres
mars_hemisphere_url = 'https://marshemispheres.com/'
browser.visit(mars_hemisphere_url)
time.sleep(1)
soup = bs(browser.html, 'html.parser')

In [ ]:
links = soup.find_all('a', class_='itemLink')

image_data = []
for l in links:
    try:
        browser.visit(mars_hemisphere_url + l['href'])
        time.sleep(1)
        soup = bs(browser.html, 'html.parser')
        hemi_title = soup.find('h2', class_='title')
        hemi_link = soup.find('img', class_='wide-image')['src']
        my_dict = {'title': hemi_title.get_text(), 'img_url': hemi_link}
        image_data.append(my_dict)
    except TypeError:
        print('Error')

In [15]:
hemisphere_image_urls = []
for element in image_data:
    if element not in hemisphere_image_urls:
        hemisphere_image_urls.append(element)
        
hemisphere_image_urls

[{'title': 'Cerberus Hemisphere Enhanced',
  'img_url': 'images/f5e372a36edfa389625da6d0cc25d905_cerberus_enhanced.tif_full.jpg'},
 {'title': 'Schiaparelli Hemisphere Enhanced',
  'img_url': 'images/3778f7b43bbbc89d6e3cfabb3613ba93_schiaparelli_enhanced.tif_full.jpg'},
 {'title': 'Syrtis Major Hemisphere Enhanced',
  'img_url': 'images/555e6403a6ddd7ba16ddb0e471cadcf7_syrtis_major_enhanced.tif_full.jpg'},
 {'title': 'Valles Marineris Hemisphere Enhanced',
  'img_url': 'images/b3c7c6c9138f57b4756be9b9c43e3a48_valles_marineris_enhanced.tif_full.jpg'}]

In [16]:
# Close browser
browser.quit()